<a href="https://colab.research.google.com/github/akshraj-netizen/Modeling-of-Electrical-Conductivity-of-Nanocomposites/blob/main/quantitative_model_for_electrical_conductivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt

# Step 1: Data Generation
# Generate simulated data with required multiplication factor to be in respective ranges
data = pd.DataFrame({
    'CNT_length': np.random.weibull(a=2.4, size=1000) * 1e-5,
    'Aspect_ratio': np.random.uniform(50, 200, size=1000),
    'CNT_conductivity': np.random.lognormal(mean=4, sigma=1, size=1000) * 1e-9,
    'Volume_fraction': np.random.uniform(0.01, 0.1, size=1000),
    'Electrical_conductivity': np.random.uniform(10, 100, size=1000) * 1e-6
})

# Save the dataset to a CSV file
data.to_csv('simulated_dataset_final.csv', index=False)
print("Data has been saved to 'simulated_dataset_final.csv'")

# Step 2: Data Preparation
# Split features and target
X = data[['CNT_length', 'Aspect_ratio', 'CNT_conductivity', 'Volume_fraction']]
y = data['Electrical_conductivity']

# Normalize features and target
scaler_X = MinMaxScaler()
X_normalized = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_normalized = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_normalized, test_size=0.2, random_state=42)

# Step 3: Model Training
# Random Forest
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# XGBoost
xgb = XGBRegressor(n_estimators=300, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

# ANN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
y_pred_ann = model.predict(X_test).flatten()

# Step 4: Inverse Transform Predictions
# Convert predictions back to original scale
y_pred_rf_original = scaler_y.inverse_transform(y_pred_rf.reshape(-1, 1)).flatten()
y_pred_xgb_original = scaler_y.inverse_transform(y_pred_xgb.reshape(-1, 1)).flatten()
y_pred_ann_original = scaler_y.inverse_transform(y_pred_ann.reshape(-1, 1)).flatten()

Data has been saved to 'simulated_dataset_final.csv'


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [ ]:
# Step 5: Combine and Display Final Predictions
# Combine predictions and input features into a DataFrame
final_results = pd.DataFrame({
    'CNT_length': scaler_X.inverse_transform(X_test)[:, 0],
    'Aspect_ratio': scaler_X.inverse_transform(X_test)[:, 1],
    'CNT_conductivity': scaler_X.inverse_transform(X_test)[:, 2],
    'Volume_fraction': scaler_X.inverse_transform(X_test)[:, 3],
    'RF_Predicted_Conductivity': y_pred_rf_original,
    'XGB_Predicted_Conductivity': y_pred_xgb_original,
    'ANN_Predicted_Conductivity': y_pred_ann_original
})

print("Final Electrical Conductivity Values:")
print(final_results.head())

Final Electrical Conductivity Values:
   CNT_length  Aspect_ratio  CNT_conductivity  Volume_fraction  \
0    0.000015    142.145890      7.804750e-08         0.047094   
1    0.000013    156.213624      5.647921e-08         0.087858   
2    0.000009     50.888012      1.141554e-07         0.025379   
3    0.000011     73.156892      5.417552e-08         0.026811   
4    0.000005    192.706667      3.090373e-07         0.047830   

   RF_Predicted_Conductivity  XGB_Predicted_Conductivity  \
0                   0.000064                    0.000067   
1                   0.000059                    0.000053   
2                   0.000054                    0.000068   
3                   0.000056                    0.000061   
4                   0.000041                    0.000041   

   ANN_Predicted_Conductivity  
0                    0.000054  
1                    0.000053  
2                    0.000055  
3                    0.000050  
4                    0.000048  
